In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/RUSTEYE/dataset_cnn_v2.zip

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 80

In [3]:
train_dir = '/content/dataset_cnn_v2/train'
val_dir   = '/content/dataset_cnn_v2/valid'
test_dir = '/content/dataset_cnn_v2/test'

# Data augmentation cho train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 945 images belonging to 4 classes.
Found 202 images belonging to 4 classes.
Found 205 images belonging to 4 classes.


In [4]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
for layer in base_model.layers:
    layer.trainable = False

# Thêm các layer phía trên
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
# Compile
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Callbacks
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)

In [7]:
# Train
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stopping]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.4914 - loss: 1.1100 - val_accuracy: 0.7574 - val_loss: 0.5872
Epoch 2/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 23s 766ms/step - accuracy: 0.7847 - loss: 0.5743 - val_accuracy: 0.7772 - val_loss: 0.4965
Epoch 3/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 21s 714ms/step - accuracy: 0.7958 - loss: 0.5029 - val_accuracy: 0.8168 - val_loss: 0.4328
Epoch 4/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 43s 769ms/step - accuracy: 0.8330 - loss: 0.4269 - val_accuracy: 0.8416 - val_loss: 0.4221
Epoch 5/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 21s 701ms/step - accuracy: 0.8191 - loss: 0.4420 - val_accuracy: 0.8416 - val_loss: 0.3842
Epoch 6/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 21s 705ms/step - accuracy: 0.8833 - loss: 0.3482 - val_accuracy: 0.8564 - val_loss: 0.3472
Epoch 7/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 21s 702ms/step - accuracy: 0.8584 - loss: 0.3600 - val_accuracy: 0.8069 - val_loss: 0.3992
Epoch 8/80
30/30 ━━━━━━━━━━━━━━━━━━━━ 20s 673ms/step - accuracy: 0.8649 - loss: 0.3270 - val_accurac

In [8]:
# Load best model
from tensorflow.keras.models import load_model
best_model = load_model('best_model.keras')

In [9]:
# Đánh giá trên test set
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Lấy nhãn class
class_labels = list(test_generator.class_indices.keys())
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Accuracy theo từng class
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_labels, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.8886 - loss: 0.2602

Test Loss: 0.2450
Test Accuracy: 89.76%
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 426ms/step
Accuracy: 89.75609756097562
Precision: 90.17460505959693
Recall: 89.75609756097562
F1 Score: 89.81815970012434

Classification Report:
                precision    recall  f1-score   support

 bong-troc-son     0.7759    0.8824    0.8257        51
day-cap-ri-set     0.9636    0.9298    0.9464        57
        ri-set     0.8723    0.7885    0.8283        52
       vet-nut     1.0000    1.0000    1.0000        45

      accuracy                         0.8976       205
     macro avg     0.9030    0.9002    0.9001       205
  weighted avg     0.9017    0.8976    0.8982       205


Accuracy for each class:
bong-troc-son: 0.88
day-cap-ri-set: 0.93
ri-set: 0.79
vet-nut: 1.00
